In [16]:
import pandas as pd
import numpy as np
from gurobipy import *

bpp_data_set = [
    {'instance_name': 'Morning:6-7','c':  100, 'w': [68, 53, 52, 35, 43, 65, 75, 42, 62, 32]},
    {'instance_name': '7-8',        'c':  200, 'w': [95, 74, 73, 72, 66, 50, 125, 144, 139, 30, 119, 114, 70, 95, 72, 120, 136,67,45,55]},
    {'instance_name': '8-9',        'c':  250, 'w': [98, 98, 98, 96, 96, 94, 93, 93, 92, 91, 91, 90, 87, 86, 85, 85, 84, 84, 84, 84, 84, 83,115,88,85,88,91,93,95,110]},
    {'instance_name': '9-10',       'c':  300, 'w': [194, 193, 192, 90, 188, 87, 184, 80, 179, 76, 74, 72, 77, 157, 162, 158, 156, 56, 153, 150, 147, 140, 140,180,175,134,122,120,230,240]},
];

def lb(c, w):
    return int(math.ceil(sum(w) / c))

df = pd.DataFrame(bpp_data_set, columns=['instance_name', 'c', 'w'])
df['n'] = df['w'].apply(len)
df['wmin'] = df['w'].apply(min)
df['wmax'] = df['w'].apply(max)
df['lb'] = df.apply(lambda x: lb(x['c'], x['w']), axis=1)
display(df)

,instance_name,c,w,n,wmin,wmax,lb
0,Morning:6-7,100,"[68, 53, 52, 35, 43, 65, 75, 42, 62, 32]",10,32,75,6
1,7-8,200,"[95, 74, 73, 72, 66, 50, 125, 144, 139, 30, 11...",20,30,144,9
2,8-9,250,"[98, 98, 98, 96, 96, 94, 93, 93, 92, 91, 91, 9...",30,83,115,11
3,9-10,300,"[194, 193, 192, 90, 188, 87, 184, 80, 179, 76,...",30,56,240,15


In [17]:
def model_bpp(c, w, UB=None, bin_for_item=None, LogToConsole=True, TimeLimit=30):
    n = len(w)
    LB = lb(c, w)
    if UB == None:
        UB = n
    if LogToConsole:
        print('c =', c, '| n =', n, '| LB =', LB, '| UB =', UB)
    model = Model()
    model.params.LogToConsole = LogToConsole
    model.params.TimeLimit = TimeLimit # seconds
    x = model.addVars(n, UB, vtype=GRB.BINARY)
    y = model.addVars(UB, vtype=GRB.BINARY)
    model.setObjective(quicksum(y[j] for j in range(UB)), GRB.MINIMIZE) # minimize the number of bins used
    model.addConstrs(quicksum(x[i,j] for j in range(UB)) == 1 for i in range(n)) # each item in exactly one bin
    model.addConstrs(quicksum(w[i] * x[i,j] for i in range(n)) <= c * y[j] for j in range(UB))
                                                                  # limit total weight in each bin; also, link $x_{ij}$ with $y_j$
    if bin_for_item != None:
        for i in range(n):
            x[i, bin_for_item[i]].start = 1
    model.optimize()
    bin_for_item = [-1 for i in range(n)]
    for i in range(n):
        for j in range(UB):
            if x[i,j].X > 0.5:
                bin_for_item[i] = j
    return model.ObjVal, model.ObjBound, bin_for_item

In [18]:
def heur_FFD(c, w): # first-fit-decreasing heuristic
    n = len(w)
    order = sorted([i for i in range(n)], key=lambda i: w[i], reverse=True) # sort by decreasing weights
    bin_for_item = [-1 for i in range(n)]
    bin_space = []
    for i in order:
        for j in range(len(bin_space)): # pack in the first bin in which the item fits
            if w[i] <= bin_space[j]:
                bin_for_item[i] = j
                bin_space[j] -= w[i]
                break
        if bin_for_item[i] < 0: # if no bin can accomodate the item, open a new bin
            j = len(bin_space)
            bin_for_item[i] = j
            bin_space.append(c - w[i])
    n_bins = len(bin_space)
    return n_bins, bin_for_item

In [19]:
from time import time

for idx, bpp_data in enumerate(bpp_data_set):
    t_start = time()
    c, w = bpp_data['c'], bpp_data['w']
    n_bins, bin_for_item = heur_FFD(c, w)
    n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, n_bins, bin_for_item, LogToConsole=False, TimeLimit=10)
    t_end = time()
    print(idx, n_bins, n_bins_lb, t_end - t_start)

0 6.0 6.0 0.010004520416259766
1 10.0 10.0 0.06301474571228027
2 15.0 15.0 0.05401420593261719
3 16.0 16.0 0.06201457977294922


In [20]:
for idx, bpp_data in enumerate(bpp_data_set):
    c, w = bpp_data['c'], bpp_data['w']
    n_bins, n_bins_lb, bin_for_item = model_bpp(c, w, LogToConsole=False, TimeLimit=5)
    print(idx, n_bins, n_bins_lb)

0 6.0 6.0
1 10.0 10.0
2 15.0 15.0
3 16.0 16.0
